In [1]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import torch
import random
import copy
import sys
import os
import time
import argparse
import json
import numpy as np
import numpy.linalg as la
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from scipy.sparse import csgraph
from torch.backends import cudnn
from torch.optim import lr_scheduler
from utils import *
from graphConvolution import *

#hyperparameters
num_node = 2708
num_coreset = 140
hidden_size = 128
dmax = np.ones(num_node)
gamma = 1
radium = 0.05

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid,bias=True)
        self.gc2 = GraphConvolution(nhid, nclass,bias=True)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return x
   
def train(epoch, model,record):

    model.train()
    optimizer.zero_grad()
    output = model(features_GCN, adj)
    loss_train = F.cross_entropy(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    model.eval()
    output = model(features_GCN, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    record[acc_val.item()] = acc_test.item()
    
    
def get_receptive_fields_dense(cur_neighbors, selected_node, weighted_score): 
    receptive_vector=((cur_neighbors+adj_matrix2[selected_node])!=0)+0
    count=weighted_score.dot(receptive_vector)
    return count

def get_current_neighbors_dense(cur_nodes):
    if np.array(cur_nodes).shape[0]==0:
        return 0
    neighbors=(adj_matrix2[list(cur_nodes)].sum(axis=0)!=0)+0
    return neighbors

def get_max_nnd_node_dense(idx_used,high_score_nodes,min_distance): 
    max_receptive_node = 0
    max_total_score = 0
    cur_neighbors=get_current_neighbors_dense(idx_used)
    for node in high_score_nodes:
        receptive_field=get_receptive_fields_dense(cur_neighbors,node,num_ones)
        node_distance = distance_aax[node,:]
        node_distance = np.where(node_distance<min_distance,node_distance,min_distance)
        node_distance = dmax - node_distance
        distance_score = node_distance.dot(num_ones)
        total_score = receptive_field/num_node+gamma*distance_score/num_node
        if total_score > max_total_score:
            max_total_score = total_score
            max_receptive_node = node        
    return max_receptive_node

def aug_normalized_adjacency(adj):
    adj = adj + sp.eye(adj.shape[0])
    adj = sp.coo_matrix(adj)
    row_sum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(row_sum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()


def compute_distance(_i,_j):
    return la.norm(features_aax[_i,:]-features_aax[_j,:])
#read dataset
adj, features, labels, idx_train, idx_val, idx_test = load_data(dataset="cora")
features_GCN = copy.deepcopy(features)
features_GCN = torch.FloatTensor(features_GCN).cuda()

num_zeros = np.zeros(num_node)
num_ones = np.ones(num_node)
idx_val = list(idx_val.cpu())
idx_test = list(idx_test.cpu())
idx_avaliable = list()
for i in range(num_node):
    if i not in idx_val and i not in idx_test:
        idx_avaliable.append(i)

#compute normalized distance
adj = aug_normalized_adjacency(adj)
adj_matrix = torch.FloatTensor(adj.todense()).cuda()
adj_matrix2 = torch.mm(adj_matrix,adj_matrix).cuda()
features = features.cuda()
features_aax = np.array(torch.mm(adj_matrix2,features).cpu())
adj_matrix2 = np.array(adj_matrix2.cpu())

distance_aax = np.zeros((num_node,num_node))
for i in range(num_node-1):
    for j in range(i+1,num_node):
        distance_aax[i][j] = compute_distance(i,j)
        distance_aax[j][i] = distance_aax[i][j]
dis_range = np.max(distance_aax) - np.min(distance_aax)
distance_aax = (distance_aax - np.min(distance_aax))/dis_range

print('GRAIN(NN-D) node selection start')
#chooose node
min_distance = np.ones(num_node)
idx_train_nnd = []
idx_avaliable_temp = copy.deepcopy(idx_avaliable)
count = 0
while True:
    max_reception_node = get_max_nnd_node_dense(idx_train_nnd,idx_avaliable_temp,min_distance) 
    idx_train_nnd.append(max_reception_node) 
    idx_avaliable.remove(max_reception_node)
    idx_avaliable_temp.remove(max_reception_node)
    count += 1
    max_node_distance = distance_aax[max_reception_node,:]
    min_distance = np.where(min_distance<max_node_distance,min_distance,max_node_distance)
    if count >= num_coreset:
        break

print('GRAIN(NN-D) node selection finished')

print('GRAIN(ball-D) node selection start')
balls = np.zeros((num_node,num_node))
balls_dict=dict()
covered_balls = set()
for i in range(num_node):
    for j in range(num_node):
        if distance_aax[i][j] <= radium:
            balls[i][j]=1

idx_avaliable_tmp = copy.deepcopy(idx_avaliable)
for node in idx_avaliable_tmp:
    neighbors_tmp = get_current_neighbors_dense([node])
    neighbors_tmp = neighbors_tmp[:,np.newaxis]
    dot_result = np.matmul(balls,neighbors_tmp).T
    tmp_set = set()
    for i in range(num_node):
        if dot_result[0,i]!=0:
            tmp_set.add(i)
    balls_dict[node]=tmp_set

#choose the node
count = 0
idx_train_balld = []
while True: 
    ball_num_max = 0
    node_max = 0
    for node in idx_avaliable_tmp:
        tmp_num = len(covered_balls.union(balls_dict[node]))
        if tmp_num > ball_num_max:
            ball_num_max = tmp_num
            node_max = node
    res_ball_num = num_node - ball_num_max
    count+=1
    idx_train_balld.append(node_max)
    idx_avaliable_tmp.remove(node_max)
    covered_balls = covered_balls.union(balls_dict[node_max])
    if count >= num_coreset or res_ball_num==0:
        break

print('GRAIN(ball-D) node selection finished')


adj = sparse_mx_to_torch_sparse_tensor(adj).float().cuda()
labels = labels.cuda()
idx_val = torch.LongTensor(idx_val).cuda()
idx_test = torch.LongTensor(idx_test).cuda()

print('xxxxxxxxxx GRAIN(NN-D) Evaluation begin  xxxxxxxxxx')
idx_train = torch.LongTensor(idx_train_nnd).cuda()
record = {}
model = GCN(nfeat=features_GCN.shape[1],
        nhid=hidden_size,
        nclass=labels.max().item() + 1,
        dropout=0.85)
model.cuda()
optimizer = optim.Adam(model.parameters(),
                       lr=0.05, weight_decay=5e-4)
for epoch in range(400):
    train(epoch,model,record)

bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)
print('xxxxxxxxxx GRAIN(NN-D) Evaluation end  xxxxxxxxxx')


print('xxxxxxxxxx GRAIN(ball-D) Evaluation begin  xxxxxxxxxx')
idx_train = torch.LongTensor(idx_train_balld).cuda()
record = {}
model = GCN(nfeat=features_GCN.shape[1],
        nhid=hidden_size,
        nclass=labels.max().item() + 1,
        dropout=0.85)
model.cuda()
optimizer = optim.Adam(model.parameters(),
                       lr=0.05, weight_decay=5e-4)
for epoch in range(400):
    train(epoch,model,record)

bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)
print('xxxxxxxxxx GRAIN(ball-D) Evaluation end  xxxxxxxxxx')


[STEP 1]: Upload cora dataset.
| # of nodes : 2708
| # of edges : 5278.0
| # of features : 1433
| # of clases   : 7
| # of train set : 140
| # of val set   : 500
| # of test set  : 1000
GRAIN(NN-D) node selection start
GRAIN(NN-D) node selection finished
GRAIN(ball-D) node selection start
GRAIN(ball-D) node selection finished
xxxxxxxxxx GRAIN(NN-D) Evaluation begin  xxxxxxxxxx
0.829 0.833
0.8240000000000001 0.825
0.819 0.817
0.815 0.809
0.81 0.799
0.808 0.81
0.806 0.812
0.804 0.807
0.802 0.81
0.8 0.804
xxxxxxxxxx GRAIN(NN-D) Evaluation end  xxxxxxxxxx
xxxxxxxxxx GRAIN(ball-D) Evaluation begin  xxxxxxxxxx
0.838 0.842
0.834 0.821
0.8240000000000001 0.823
0.822 0.817
0.8200000000000001 0.824
0.818 0.811
0.816 0.817
0.814 0.805
0.812 0.823
0.81 0.788
xxxxxxxxxx GRAIN(ball-D) Evaluation end  xxxxxxxxxx
